![alt text](virlow.png "Virlow")



# Virlow Speech-to-Text API with JupyterLab

The Virlow API can be used to quickly convert pre-recorded audio files and real-time audio streams into accurate text transcriptions. This Jupyter notebook/lab allows you to easily use the Virlow API in your application.

## Install required packages

<div class="alert alert-block alert-info">
<b>Info:</b> Tested on macOS.
</div>

1. `pip install torchaudio ipywebrtc`
1. `jupyter nbextension enable --py widgetsnbextension`
1. Install ffmpeg - https://www.ffmpeg.org/download.html

## Record an audio file

In [ ]:
from ipywebrtc import AudioRecorder, CameraStream
import torchaudio
from IPython.display import Audio

In [ ]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav audio.wav -y -hide_banner -loglevel panic
sig, sr = torchaudio.load("audio.wav")
print(sig.shape)
Audio(data=sig, rate=sr)

## Transcribe your audio file

1. Get your API Key in the Virlow Console.

2. Follow the intrudctions on how to create an API Key at https://docs.voice.virlow.com/reference/authentication-1

In [ ]:
API_KEY="YOUR_API_KEY"

In [ ]:
import requests

url = "https://api.voice.virlow.com/v1-beta/transcript?x-api-key=" + API_KEY

payload = {
    'dual_channel': 'false',
    'punctuate': 'true',
    'webhook_url': '',
    'speaker_diarization': 'false',
    'language': 'enUs',
    'short_hand_notes': 'true',
    'tldr': 'true',
    'custom': 'YOUR_VALUE'}

files = [('audio',('audio.wav',open('audio.wav','rb'),'audio/wav'))]

headers = {}

response = requests.request("POST", url, headers=headers, data=payload, files=files)

JOB_ID = response.json()["id"]

print(response.text)

## Get the transcription results



<div class="alert alert-block alert-info">
<b>Info:</b> To provide you an affordable Speech-to-Text API it will take a few minutes for the audio file to be processed. It will take between 1 to 5 mintues before the file starts to process.
</div>


In [ ]:
import json

url = "https://api.voice.virlow.com/v1-beta/transcript/status?x-api-key=" + API_KEY + "&id=" + JOB_ID

response = requests.request("GET", url)

json_data = json.loads(response.text)

In [ ]:
print(json.dumps(json_data, indent=4, sort_keys=True))